In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/Colab Notebooks/pygaggle-master/

/content/drive/MyDrive/Colab Notebooks/pygaggle-master


In [5]:
ls

data/            indexes/     models/     requirements.txt  setup.py
docs/            LICENSE      notebooks/  runs/             tests/
environment.yml  logs/        pygaggle/   scripts/          tools/
experiments/     MANIFEST.IN  README.md   setup.cfg


In [6]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment


In [8]:
cd /content/drive/MyDrive/Colab Notebooks/pygaggle-master/

/content/drive/MyDrive/Colab Notebooks/pygaggle-master


test top 100을 Bert로 rerank하고 재정렬해서 JSON으로 뽑아내는 코드

In [45]:
import json

queryy=[]
#test set의 경우에는 list 사이즈 [3610][100][3]

file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_test_top100_all.json'    #test set 이용
#file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_dev_top100_all.json'    #dev set 이용

with open(file_path, 'r', encoding='utf-8') as file:
      data = json.load(file)
      data_size=len(data)
      corpus = [[['a' for _ in range(4)] for _ in range(100)] for _ in range(data_size)]
      for i in range(len(data)):
       queryy.append(data[i]["question"])
       for j in range(100):
        corpus[i][j][0]=data[i]["ctxs"][j]["id"]
        corpus[i][j][1]=data[i]["ctxs"][j]["title"]
        corpus[i][j][2]=data[i]["ctxs"][j]["text"]
        corpus[i][j][3]=data[i]["ctxs"][j]["has_answer"]

print(data_size)
#print(type(corpus[0][99][2]))
#print(corpus[0][99][1])
#print(queryy[1])
#print(data[0]["ctxs"][2]["score"])

3610


In [46]:
#Bert 모델 이용할때 이거 실행하면됨
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoBERT

reranker =  MonoBERT()

In [ ]:
tot_size=0 #100개의 passage중 has_answer가 하나라도 true 질문의 개수
accurate=0 #100개의 passage중 has_answer가 true인 query에 대해 reranked된 top-k에 has_answer가 하나라도 true인 개수

for q in range(0,1805): #(앞 1805개), range(data_size):
  #print(queryy[q])
  print(q)
  query = Query(queryy[q])
  passages=corpus[q] 

  texts = [ Text(p[2], {'id': p[0],'title':p[1],'text':p[2],'has_answer' : p[3]}, 0) for p in passages]

 #기존 passage의 has_answer가 모두 false인지 아닌지 보는것(accuracy계산에 활용), flag=1이면 has_answer가 하나라도 있는거임
  flag=0
  for a in range(100):
    if texts[a].metadata["has_answer"]==1:
      flag=1
      tot_size+=1
      break

  reranked = reranker.rerank(query, texts)

  if flag==1:
   for i in range(0, 10):
       if reranked[i].metadata["has_answer"]==1:
         accurate+=1
         break

  #json 파일 값 변경
  for k in range(100):
    data[q]["ctxs"][k]["id"]=reranked[k].metadata["id"]
    data[q]["ctxs"][k]["title"]=reranked[k].metadata["title"]
    data[q]["ctxs"][k]["text"]=reranked[k].metadata["text"]
    data[q]["ctxs"][k]["score"]=str(reranked[k].score) 
    data[q]["ctxs"][k]["has_answer"]=reranked[k].metadata["has_answer"]

accuracy=accurate/tot_size
print(f'Accuracy : {accuracy:.4f}')

with open('NQ_test_top100_reranked_bert_1', 'w') as file:
      json.dump(data[0:1805], file)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


In [ ]:
tot_size=0 #100개의 passage중 has_answer가 하나라도 true 질문의 개수
accurate=0 #100개의 passage중 has_answer가 true인 query에 대해 reranked된 top-k에 has_answer가 하나라도 true인 개수

for q in range(1805,3610): #(뒤 1805개), range(data_size):
  #print(queryy[q])
  print(q)
  query = Query(queryy[q])
  passages=corpus[q]

  texts = [ Text(p[2], {'id': p[0],'title':p[1],'text':p[2],'has_answer' : p[3]}, 0) for p in passages]

 #기존 passage의 has_answer가 모두 false인지 아닌지 보는것(accuracy계산에 활용), flag=1이면 has_answer가 하나라도 있는거임
  flag=0
  for a in range(100):
    if texts[a].metadata["has_answer"]==1:
      flag=1
      tot_size+=1
      break

  reranked = reranker.rerank(query, texts)

  if flag==1:
   for i in range(0, 10):
       if reranked[i].metadata["has_answer"]==1:
         accurate+=1
         break

  #json 파일 값 변경
  for k in range(100):
    data[q]["ctxs"][k]["id"]=reranked[k].metadata["id"]
    data[q]["ctxs"][k]["title"]=reranked[k].metadata["title"]
    data[q]["ctxs"][k]["text"]=reranked[k].metadata["text"]
    data[q]["ctxs"][k]["score"]=str(reranked[k].score) 
    data[q]["ctxs"][k]["has_answer"]=reranked[k].metadata["has_answer"]

accuracy=accurate/tot_size
print(f'Accuracy : {accuracy:.4f}')

with open('NQ_test_top100_reranked_bert_2', 'w') as file:
      json.dump(data[1805:3610], file)

test top 100을 T5로 rerank하고 재정렬해서 JSON으로 뽑아내는 코드

In [ ]:
import json

queryy=[]
#test set의 경우에는 list 사이즈 [3610][100][3]

file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_test_top100_all.json'    #test set 이용
#file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_dev_top100_all.json'    #dev set 이용

with open(file_path, 'r', encoding='utf-8') as file:
      data = json.load(file)
      data_size=len(data)
      corpus = [[['a' for _ in range(4)] for _ in range(100)] for _ in range(data_size)]
      for i in range(len(data)):
       queryy.append(data[i]["question"])
       for j in range(100):
        corpus[i][j][0]=data[i]["ctxs"][j]["id"]
        corpus[i][j][1]=data[i]["ctxs"][j]["title"]
        corpus[i][j][2]=data[i]["ctxs"][j]["text"]
        corpus[i][j][3]=data[i]["ctxs"][j]["has_answer"]

print(data_size)
#print(type(corpus[0][99][2]))
#print(corpus[0][99][1])
#print(queryy[1])
#print(data[0]["ctxs"][2]["score"])

In [ ]:
#T5 모델 이용
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5

reranker =  MonoT5()

In [ ]:
tot_size=0 #100개의 passage중 has_answer가 하나라도 true 질문의 개수
accurate=0 #100개의 passage중 has_answer가 true인 query에 대해 reranked된 top-k에 has_answer가 하나라도 true인 개수

for q in range(0,1805): #(앞 1805개), range(data_size):
  #print(queryy[q])
  print(q)
  query = Query(queryy[q])
  passages=corpus[q]

  texts = [ Text(p[2], {'id': p[0],'title':p[1],'text':p[2],'has_answer' : p[3]}, 0) for p in passages]

 #기존 passage의 has_answer가 모두 false인지 아닌지 보는것(accuracy계산에 활용), flag=1이면 has_answer가 하나라도 있는거임
  flag=0
  for a in range(100):
    if texts[a].metadata["has_answer"]==1:
      flag=1
      tot_size+=1
      break

  reranked = reranker.rerank(query, texts)

  if flag==1:
   for i in range(0, 10):
       if reranked[i].metadata["has_answer"]==1:
         accurate+=1
         break

  #json 파일 값 변경
  for k in range(100):
    data[q]["ctxs"][k]["id"]=reranked[k].metadata["id"]
    data[q]["ctxs"][k]["title"]=reranked[k].metadata["title"]
    data[q]["ctxs"][k]["text"]=reranked[k].metadata["text"]
    data[q]["ctxs"][k]["score"]=str(reranked[k].score) 
    data[q]["ctxs"][k]["has_answer"]=reranked[k].metadata["has_answer"]

accuracy=accurate/tot_size
print(f'Accuracy : {accuracy:.4f}')

with open('NQ_test_top100_reranked_t5_1', 'w') as file:
      json.dump(data[0:1805], file)

In [ ]:
tot_size=0 #100개의 passage중 has_answer가 하나라도 true 질문의 개수
accurate=0 #100개의 passage중 has_answer가 true인 query에 대해 reranked된 top-k에 has_answer가 하나라도 true인 개수

for q in range(1805,3610): #(뒤 1805개), range(data_size):
  #print(queryy[q])
  print(q)
  query = Query(queryy[q])
  passages=corpus[q]

  texts = [ Text(p[2], {'id': p[0],'title':p[1],'text':p[2],'has_answer' : p[3]}, 0) for p in passages]

 #기존 passage의 has_answer가 모두 false인지 아닌지 보는것(accuracy계산에 활용), flag=1이면 has_answer가 하나라도 있는거임
  flag=0
  for a in range(100):
    if texts[a].metadata["has_answer"]==1:
      flag=1
      tot_size+=1
      break

  reranked = reranker.rerank(query, texts)

  if flag==1:
   for i in range(0, 10):
       if reranked[i].metadata["has_answer"]==1:
         accurate+=1
         break

  #json 파일 값 변경
  for k in range(100):
    data[q]["ctxs"][k]["id"]=reranked[k].metadata["id"]
    data[q]["ctxs"][k]["title"]=reranked[k].metadata["title"]
    data[q]["ctxs"][k]["text"]=reranked[k].metadata["text"]
    data[q]["ctxs"][k]["score"]=str(reranked[k].score) 
    data[q]["ctxs"][k]["has_answer"]=reranked[k].metadata["has_answer"]

accuracy=accurate/tot_size
print(f'Accuracy : {accuracy:.4f}')

with open('NQ_test_top100_reranked_t5_2', 'w') as file:
      json.dump(data[1805:3610], file)